In [ ]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import evaluate,train,prepare
from models.NRMS import NRMSModel
from configs.ManualConfig import hparams

In [ ]:
hparams['name'] = 'nrms'
hparams['device'] = 'cuda:0'
hparams['onehot'] = True

In [ ]:
vocab, loaders = prepare(hparams)

In [ ]:
record = next(iter(loaders[0]))

In [ ]:
nrmsModel = NRMSModel(hparams=hparams, vocab=vocab).to(hparams['device'])

In [ ]:
nrmsModel(record)

In [ ]:
train(nrmsModel, hparams, loaders)